In [189]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

import warnings
warnings.filterwarnings('ignore')

import matplotlib.dates as mdates
from mpl_finance import candlestick_ohlc

In [190]:
import nltk
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

In [191]:
a = datetime.date(2017,01,01).strftime("%s")
a

'1483257600'

In [194]:
# load BTC data

import json
import requests

def get_data_spec(coin, date, time_period,apiKey):
    """ Query the API for 2000 units historical price data starting from "date". """
    url = "https://min-api.cryptocompare.com/data/{}?fsym={}&tsym=USD&limit=2000&toTs={}&api_key={}".format(time_period, coin, date,apiKey)
    r = requests.get(url)
    ipdata = r.json()
    return ipdata

def get_df_spec(time_period, coin, from_date, to_date):
    """ Get historical price data between two dates. If further apart than query limit then query multiple times. """
    date = to_date
    holder = []
    while date > from_date:
        # Now we use the new function to query specific coins
        data = get_data_spec(coin, date, time_period,apiKey) 
        holder.append(pd.DataFrame(data['Data']))
        date = data['TimeFrom'] 
    df = pd.concat(holder, axis = 0)
    df = df[df['time']>from_date]
    df['time'] = pd.to_datetime(df['time'], unit='s') 
    df.set_index('time', inplace=True)
    df.sort_index(ascending=False, inplace=True)
    # And just keep the close price, with the column heading as the name of the coin. 
    #df.rename(columns={'close':coin}, inplace=True)
    return df

# def load_data (fromtime,totime = 'now',cointype,frequency):
#     
#     if totime == 'now':
#         toT = datetime.now().strftime("%s")
#     url = 'https://min-api.cryptocompare.com/data/histo' + frequency
#     payload = {
#         "api_key": apiKey,
#         "fsym": cointype,
#         "tsym": "USD",
#         "limit": 2000,
#         "toTs": totime
#     }

# res_BTC = requests.get(url, params=payload)
# hist_BTC = pd.DataFrame(json.loads(res_BTC.content)['Data'])
# hist_BTC = hist_BTC.set_index('time')
# hist_BTC.index = pd.to_datetime(hist_BTC.index, unit='s')

# parameters
fromdate = int(datetime.date(2017,01,01).strftime("%s"))
todate = datetime.date(2019,04,01).strftime("%s") #today
coin = 'BTC'
timeperiod = 'histoday'
apiKey = 'bf7c04a024b244dea99e95798fa8e102b7c9738c0933795253c2c8f39f2d160c'

df = get_df_spec(timeperiod,coin,fromdate, todate)

In [ ]:
df.tail()

,close,high,low,open,volumefrom,volumeto
time,,,,,,
2019-04-01,4151.32,4164.33,4082.20,4112.69,36921.55,1.528268e+08
2019-03-31,4112.69,4129.43,4098.79,4118.13,13792.73,5.672743e+07
2019-03-30,4118.13,4146.88,4066.87,4111.38,24906.26,1.024005e+08
2019-03-29,4111.38,4122.89,4031.22,4036.45,41843.31,1.709151e+08
2019-03-28,4036.45,4046.44,4014.69,4045.98,25343.13,1.022963e+08


In [80]:
# get price change log return
target_col = 'close'
s_test = hist_BTC[target_col]
log_return = np.log(s_test/s_test.shift())
hist_BTC['price change'] = log_return

# feature extraction from OHLC
'''close/open: since cryprocurrencies are continuously trading, this is the same as price change above
close price of t-1 is the same as open price of t'''
#hist_BTC['close/open'] = np.log(hist_BTC['close']/hist_BTC['open'])
'''high/open'''
hist_BTC['high/open'] = np.log(hist_BTC['high']/hist_BTC['open'])
'''low/open'''
hist_BTC['low/open'] = np.log(hist_BTC['low']/hist_BTC['open'])
'''close/high'''
hist_BTC['close/high'] = np.log(hist_BTC['close']/hist_BTC['high'])
'''close/low'''
hist_BTC['close/low'] = np.log(hist_BTC['close']/hist_BTC['low'])
hist_BTC = hist_BTC.drop(['close','high','low','open','volumefrom'],axis=1)
hist_BTC.rename(columns = {'volumeto':'volumn'},inplace=True)
hist_BTC = hist_BTC.dropna()

In [157]:
hist_BTC.tail()

,volumn,price change,high/open,low/open,close/high,close/low
time,,,,,,
2019-03-30,4.433347e+07,-0.007786,0.018252,-0.022621,-0.026038,0.014835
2019-03-31,1.878025e+07,-0.006231,0.000349,-0.013349,-0.006579,0.007118
2019-04-01,4.717548e+07,-0.003870,0.010757,-0.016426,-0.014626,0.012556
2019-04-02,1.997890e+08,0.150751,0.167705,-0.001764,-0.016954,0.152515
2019-04-03,2.408487e+08,0.077329,0.086713,-0.009993,-0.009384,0.087323


In [153]:
hist_BTC.index[2].strftime("%s")

'1439103600'

In [154]:
# news data
import datetime

url = 'https://min-api.cryptocompare.com/data/v2/news/'

payload = {
    "api_key": apiKey,
    "lang": "EN",
    'categories':'BTC',
    "lTs" :"1506409200"
}

# 86400

# retrieve news data
res_BTC = requests.get(url, params=payload)
news_BTC = pd.DataFrame(json.loads(res_BTC.content)['Data'])
news_BTC['time'] = news_BTC['published_on'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime("%Y-%m-%d"))
news_BTC = news_BTC.set_index('time')

# get vader sentiment score on news content
news_BTC['news_content'] = news_BTC['title'] + news_BTC['body']
news_BTC['vader_polarity'] = news_BTC['news_content'].apply(lambda x: sia.polarity_scores(x))
news_BTC['vader_compound'] = news_BTC['vader_polarity'].apply(lambda x: x['compound'])
news_BTC['vader_neg'] = news_BTC['vader_polarity'].apply(lambda x: x['neg'])
news_BTC['vader_neu'] = news_BTC['vader_polarity'].apply(lambda x: x['neu'])
news_BTC['vader_pos'] = news_BTC['vader_polarity'].apply(lambda x: x['pos'])

In [156]:
news_BTC.head()

,body,categories,downvotes,guid,id,imageurl,lang,published_on,source,source_info,tags,title,upvotes,url,news_content,vader_polarity,vader_compound,vader_neg,vader_neu,vader_pos
time,,,,,,,,,,,,,,,,,,,,
2017-09-25,As inflation takes a bigger bite out of payche...,BTC|Trading,0,https://cointelegraph.com/news/bitcoin-trading...,256,https://images.cryptocompare.com/news/default/...,EN,1506394824,cointelegraph,"{u'lang': u'EN', u'name': u'CoinTelegraph', u'...",Bitcoin|Zimbabwe|Venezuela,Bitcoin Trading at 85% Premium in Zimbabwe - P...,0,https://cointelegraph.com/news/bitcoin-trading...,Bitcoin Trading at 85% Premium in Zimbabwe - P...,"{u'neg': 0.083, u'neu': 0.917, u'pos': 0.0, u'...",-0.4215,0.083,0.917,0.000
2017-09-25,Bitcoin is a new and intriguing protocol that ...,BTC|Blockchain,0,https://news.bitcoin.com/?p=61393,254,https://images.cryptocompare.com/news/bitcoin....,EN,1506393012,bitcoin.com,"{u'lang': u'EN', u'name': u'Bitcoin.com', u'im...",Reviews|Bitbonkers|Bitcoin network|Bitlisten|B...,A Glimpse at Six Unique Cryptocurrency Visuali...,0,https://news.bitcoin.com/a-glimpse-at-six-uniq...,A Glimpse at Six Unique Cryptocurrency Visuali...,"{u'neg': 0.0, u'neu': 0.966, u'pos': 0.034, u'...",0.3612,0.000,0.966,0.034
2017-09-25,Swapper is an automated blockchain based servi...,BTC|Regulation|Exchange|Blockchain|Sponsored,0,http://www.newsbtc.com/?p=49208,249,https://images.cryptocompare.com/news/newsbtc/...,EN,1506385874,newsbtc,"{u'lang': u'EN', u'name': u'NewsBTC', u'img': ...",Press Release|1031|bitcoin|cryptocurrency|swapper,Swapper.io Seeks to Protect Bitcoin Users from...,0,http://www.newsbtc.com/2017/09/26/swapper-io-s...,Swapper.io Seeks to Protect Bitcoin Users from...,"{u'neg': 0.053, u'neu': 0.777, u'pos': 0.17, u...",0.8885,0.053,0.777,0.170
2017-09-25,If you were a real early hoarder of Bitcoin wh...,BTC|Market,0,http://www.newsbtc.com/?p=49186,246,https://images.cryptocompare.com/news/newsbtc/...,EN,1506383036,newsbtc,"{u'lang': u'EN', u'name': u'NewsBTC', u'img': ...",Bitcoin News|News|bitcoin|cryptocurrency|mansi...,Bitcoin Can Now Buy a Mansion in Moscow Suburbs,0,http://www.newsbtc.com/2017/09/26/bitcoin-mans...,Bitcoin Can Now Buy a Mansion in Moscow Suburb...,"{u'neg': 0.05, u'neu': 0.904, u'pos': 0.046, u...",0.1531,0.050,0.904,0.046
2017-09-25,Hyperinflation in Venezuela has made a reality...,BTC|Mining,0,http://bitcoinist.com/?p=49906,243,https://images.cryptocompare.com/news/bitcoini...,EN,1506380448,bitcoinist,"{u'lang': u'EN', u'name': u'Bitcoinist', u'img...",Bitcoin|Mining|News|News teaser|bitcoin|bitcoi...,Venezuelan Hyperinflation Makes Bitcoin An Ide...,0,https://bitcoinist.com/venezuelan-hyperinflati...,Venezuelan Hyperinflation Makes Bitcoin An Ide...,"{u'neg': 0.0, u'neu': 0.833, u'pos': 0.167, u'...",0.7783,0.000,0.833,0.167


In [134]:
import datetime

d = datetime.date(2017,5,27)
d.strftime("%s")

'1495868400'

'1506409200'

In [84]:
# social data

import datetime

url = 'https://min-api.cryptocompare.com/data/social/coin/histo/day'

payload = {
    "api_key": apiKey,
    "coinID": "1182",
    'aggregate':1,
    'limit':100,
    'toTs':1543622400
}

res_BTC = requests.get(url, params=payload)
social_BTC = pd.DataFrame(json.loads(res_BTC.content)['Data'])
social_BTC['time'] = social_BTC['time'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime("%Y-%m-%d"))
social_BTC = social_BTC.set_index('time')

In [91]:
#social_BTC.head()